In [24]:
import pandas as pd
import time
import requests
from datetime import datetime
import pandas as pd
from key import api_key_secret

address="0x297B0BC8c5243fB7Ab7072B855d57B1b20Cbb964"


def convert_timestamp(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    formatted_dt = dt.strftime("%d-%m-%y %H:%M")
    return formatted_dt

def get_normal_transactions(address, api_key, page=1, offset=100):
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&page={page}&offset={offset}&sort=desc&apikey={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data['message'] == 'OK':
            transactions = data['result']
            return transactions
    return None

def get_internal_transactions(address, api_key, page=1, offset=100):
    url = f"https://api.etherscan.io/api?module=account&action=txlistinternal&address={address}&startblock=0&endblock=99999999&page={page}&offset={offset}&sort=desc&apikey={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data['message'] == 'OK':
            transactions = data['result']
            return transactions
    return None

def get_latest_erc721_transactions(address, api_key, page=1, offset=100):
    url = f"https://api.etherscan.io/api?module=account&action=tokennfttx&address={address}&startblock=0&endblock=99999999&page={page}&offset={offset}&sort=desc&apikey={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data['message'] == 'OK':
            transactions = data['result']
            return transactions
    return None

def get_latest_erc20_transactions(address, api_key, page=1, offset=100):
    url = f"https://api.etherscan.io/api?module=account&action=tokentx&address={address}&startblock=0&endblock=999999999&sort=desc&page={page}&offset={offset}&apikey={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data['message'] == 'OK':
            transactions = data['result']
            return transactions
    return None

def get_latest_erc1155_transactions(address, api_key, page=1, offset=100):
    url = f"https://api.etherscan.io/api?module=account&action=token1155tx&address={address}&startblock=0&endblock=999999999&sort=desc&page={page}&offset={offset}&apikey={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data['message'] == 'OK':
            transactions = data['result']
            return transactions
    return None

api_key = api_key_secret


ethereum_transactions = []
normal_transactions = []
internal_transactions = []
erc20_transactions = []
erc721_transactions = []
erc1155_transactions = []

page = 1
offset = 100
while True:
    transactions = get_normal_transactions(address, api_key, page=page, offset=offset)
    if not transactions or len(transactions) == 0:
        break
    normal_transactions.extend(transactions)
    page += 1


page = 1
while True:
    transactions = get_internal_transactions(address, api_key, page=page, offset=offset)
    if not transactions or len(transactions) == 0:
        break
    internal_transactions.extend(transactions)
    page += 1

page = 1
while True:
    transactions = get_latest_erc721_transactions(address, api_key, page=page, offset=offset)
    if not transactions or len(transactions) == 0:
        break
    erc721_transactions.extend(transactions)
    page += 1

page = 1
while True:
    transactions = get_latest_erc20_transactions(address, api_key, page=page, offset=offset)
    if not transactions or len(transactions) == 0:
        break
    erc20_transactions.extend(transactions)
    page += 1

page = 1
while True:
    transactions = get_latest_erc1155_transactions(address, api_key, page=page, offset=offset)
    if not transactions or len(transactions) == 0:
        break
    erc1155_transactions.extend(transactions)
    page += 1

normal_df = pd.DataFrame(normal_transactions)
internal_df = pd.DataFrame(internal_transactions)
erc721_df = pd.DataFrame(erc721_transactions)
erc1155_df = pd.DataFrame(erc1155_transactions)
erc20_df = pd.DataFrame(erc20_transactions)

In [25]:
normal_df

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName
0,19705221,1713719891,0xdbc6e98182329ff61971b03322a5f4953ce8ffdede29...,351367,0xc2aadf7e6b02dcd74590873c24837133cff5e7b71c8b...,137,0x1ab4973a48dc892cd9971ece8e01dcc7688f8f23,0x297b0bc8c5243fb7ab7072b855d57b1b20cbb964,14400000000000000,100000,21686106323,0,1,0x,,12652418,21000,3,0x,


In [26]:
internal_df

""


In [27]:
erc721_df

""


In [28]:
erc1155_df

""


In [29]:
erc20_df

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,19477817,1710960791,0xad10c86c4c14391fd135169675f7cd7f30113f2aee61...,237686,0x5b25814d85ba99249f3f160e16ca635acb99f348d390...,0x1ab4973a48dc892cd9971ece8e01dcc7688f8f23,0xdac17f958d2ee523a2206206994597c13d831ec7,0x297b0bc8c5243fb7ab7072b855d57b1b20cbb964,435000000,Tether USD,USDT,6,179,100000,37008460445,63209,14628226,deprecated,227410


In [30]:
normal_df['tokenName'] = 'Ethereum'
normal_df['ToT']='normal'
erc20_df['ToT']='erc20'
erc721_df['ToT']='erc721'
erc1155_df['ToT']='erc1155'
internal_df['ToT']='internal'
normal_df['tokenSymbol'] = 'ETH'
normal_df['tokenDecimal'] = 18
internal_df['tokenDecimal'] = 18
erc721_df["isError"] = 0
erc1155_df['isError'] = 0
erc20_df['isError'] = 0
normal_df


,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,...,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,tokenName,ToT,tokenSymbol,tokenDecimal
0,19705221,1713719891,0xdbc6e98182329ff61971b03322a5f4953ce8ffdede29...,351367,0xc2aadf7e6b02dcd74590873c24837133cff5e7b71c8b...,137,0x1ab4973a48dc892cd9971ece8e01dcc7688f8f23,0x297b0bc8c5243fb7ab7072b855d57b1b20cbb964,14400000000000000,100000,...,,12652418,21000,3,0x,,Ethereum,normal,ETH,18


In [31]:
combined_df = pd.concat([normal_df, internal_df, erc721_df, erc20_df, erc1155_df], ignore_index=True).sort_values("timeStamp", ascending=True)
combined_df.reset_index(drop=True, inplace=True)


In [32]:
normal_df.to_csv("normaldf.csv", index=False)
internal_df.to_csv("internaldf.csv", index=False)
erc721_df.to_csv("erc721df.csv", index=False)
erc1155_df.to_csv("erc1155df.csv", index=False)
erc20_df.to_csv("erc20df.csv", index=False)

combined_df = pd.concat([normal_df, internal_df, erc721_df, erc20_df, erc1155_df], ignore_index=True).sort_values("timeStamp", ascending=True)
combined_df.reset_index(drop=True, inplace=True)
combined_df.to_csv("combineddf.csv", index=False)

In [33]:


def merge_transactions_by_hash(df_group, address, dtf):
    data = {
        "timeStamp": df_group.iloc[0]["timeStamp"],
        "ETH": 0
    }
    total_fees = 0
    fees_added = False

    contract_symbol_mapping = {}

    for _, transaction in df_group.iterrows():
        err = int(transaction["isError"])
        from_address = transaction["from"]
        to_address = transaction["to"]
        is_receiver = to_address.lower() == address.lower()
        contract_address = transaction["contractAddress"]

        if contract_address and contract_address not in contract_symbol_mapping:
            contract_symbol_mapping[contract_address] = transaction["tokenSymbol"]

        if(transaction["ToT"] != "internal"):
            gasPrice = int(transaction["gasPrice"])
            gasUsed = int(transaction["gasUsed"])
            fees = (gasPrice * gasUsed) / 10**18

        if not is_receiver and not fees_added:
            total_fees += fees
            fees_added = True

        if err:
            value = 0
        else:
            if transaction["ToT"] == "internal" or transaction["tokenSymbol"] == "ETH":
                value = int(transaction["value"]) / (10 ** int(transaction["tokenDecimal"]))
                if not is_receiver:
                    value = -value

                data["ETH"] += value

            elif transaction["ToT"] != "erc721":
                value = transaction["value"]
                if pd.isnull(value):
                    value = 0
                else:
                    value = int(value)

                if contract_address == "ETH":
                    value /= (10 ** 18)
                else:
                    if pd.isnull(transaction["tokenDecimal"]):
                        value /= (10 ** 1)
                    else:
                        value /= (10 ** int(transaction["tokenDecimal"]))

                if not is_receiver:
                    value = -value

                if contract_address in data:
                    data[contract_address] += value
                else:
                    data[contract_address] = value

    if dtf.empty:
        data["ETH"] -= total_fees
    else:
        data["ETH"] += dtf.iloc[-1]["ETH"] - total_fees

    single_transaction = pd.DataFrame(data, index=[data["timeStamp"]])
    return single_transaction, contract_symbol_mapping



def version2Processing(hashList):
    dtf = pd.DataFrame(columns=["timeStamp"])
    dtf.set_index("timeStamp", inplace=True)
    contract_symbol_mapping = {}

    for hash_value in hashList:
        dfTemp = combinedGP_df.get_group(hash_value)
        single_transaction, single_mapping = merge_transactions_by_hash(dfTemp, address, dtf)
        contract_symbol_mapping.update(single_mapping)
        dtf = pd.concat([dtf, single_transaction])
        dtf.fillna(0, inplace=True)

        if dtf.shape[0] > 1:
            eth = dtf.iloc[-1]['ETH']
            timestamp = dtf.iloc[-1]["timeStamp"]

            dtf.iloc[-1] = dtf.iloc[-2] + dtf.iloc[-1]

            dtf.at[dtf.index[-1], 'ETH'] = eth
            dtf.at[dtf.index[-1], 'timeStamp'] = timestamp

    return dtf, contract_symbol_mapping



# Ici, vous devriez charger vos données et créer un DataFrame combiné
# Exemple : combined_df = ...

combinedGP_df = combined_df.groupby("hash")
hash_min_timestamps = combinedGP_df['timeStamp'].min()
sorted_hashes = hash_min_timestamps.sort_values().index

tokenAmount_DF, contract_symbol_mapping = version2Processing(sorted_hashes)
tokenAmount_DF = tokenAmount_DF.rename(columns=contract_symbol_mapping)
tokenAmount_DF.to_csv("token2.csv", index=False)

tokenAmount_DF


,timeStamp,ETH,USDT
1710960791,1710960791,0.0000,435.0
1713719891,1713719891,0.0144,435.0


In [34]:
# Prenez la dernière ligne de tokenAmount_DF
last_row = tokenAmount_DF.iloc[-1]

# Créez un nouveau DataFrame à partir de la dernière ligne
balance_df = pd.DataFrame(last_row).reset_index()

# Renommez les colonnes pour les rendre plus compréhensibles
balance_df.columns = ['Token', 'Balance']

# Supprimez la ligne 'timeStamp'
balance_df = balance_df[balance_df['Token'] != 'timeStamp']

# Réinitialisez l'index du DataFrame
balance_df.reset_index(drop=True, inplace=True)

# Affichez le DataFrame
print(balance_df)


  Token Balance
0   ETH  0.0144
1  USDT   435.0
